In [11]:
import torch

In [ ]:
def hard_nms(box_scores, iou_threshold, top_k=-1, candidate_size=200):
    '''
    Args:
        box_scores:
        iou_threshold:
        top_k:
        candidate_size:
    Returns:
        remain:
    '''
    
    scores = box_scores[:, -1] #取box_scores中每行的最后一个元素
    boxes = box_scores[:, :-1] #取box_scores中每行前4个元素
    remain = []
    _,indexes = scores.sort(descending=True) #对scores进行降序排序
    indexes = indexes[:candidate_size] #只保留前candidate_size个候选框
    
    while len(indexes) > 0:
        current = indexes[0]
        remain.append(current.item())
        if 0 < top_k == len(len(remain)) or len(indexes) == 1:
            break
        current_box = boxes[current, :]
        indexes = indexes[1:]
        rest_boxes = boxes[indexes, :]
        iou = iou_of(rest_boxes, current_box.unsqueeze(0))
        indexes = indexes[iou <= iou_threshold]
        
    return box_scores[remain, :]
        

def area_of(left_top, right_bottle) -> torch.Tensor:
    '''
    Args:
        left_top:
        right_bottle:
    Returns:
        area:
    '''
    
    hw = torch.clamp(right_bottle - left_top, min=0.0)
    
    return hw[:,0] * hw[:,1]


def iou_of(boxes_1, boxes_2, eps=1e-5):
    '''
    Args:
        boxes_1:
        boxes_2:
        eps:
    Returns:
        iou:
    '''
    
    overlap_left_top = torch.max(boxes_1[:, :2], boxes_2[:, :2])
    overlap_rigth_bottle = torch.min(boxes_1[:, 2:], boxes_2[:, 2:])
    overlap_area = area_of(overlap_left_top, overlap_rigth_bottle)
    area1 = area_of(boxes_1[:, :2], boxes_1[:, 2:])
    area2 = area_of(boxes_2[:, :2], boxes_2[:, 2:])
    
    return overlap_area / (area1 + area2 - overlap_area + eps)